In [ ]:
import numpy as np
from tqdm import tqdm, trange
import torch
from torch import nn
from torch.utils.data import DataLoader, random_split
from transformers import AutoModelForCausalLM, AutoTokenizer

from gsm_dataset import GSMDataset, gsm_collate, gsm_prompt, sample
from biscuit import Biscuit

/home/BOSDYN/cjestin/Research/biscuit/biscuit-env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
biscuit_model = Biscuit()
checkpoint_path = f'latent_trunk_epoch_9.pth'
biscuit_model.latent_trunk.load_state_dict(torch.load(checkpoint_path))

<All keys matched successfully>

In [14]:
biscuit_model.bot_embedding = torch.load('bot_epoch_9.pt')
biscuit_model.eot_embedding = torch.load('eot_epoch_9.pt')

In [24]:
dataset = GSMDataset()

train_size = int(0.9 * len(dataset))
example_size = int(0.02 * len(dataset)) # reserve some data for few shot prompting
test_size = len(dataset) - train_size - example_size

train_dataset, example_dataset, test_dataset = random_split(dataset, [train_size, example_size, test_size])
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=gsm_collate)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=True, collate_fn=gsm_collate)

In [28]:
COT_MAX_LENGTH = 6
segments, keep_indices_lst = next(iter(train_loader))
examples = sample(example_dataset, num_samples=4)
prompt = gsm_prompt(examples)

softmax = nn.Softmax(dim=1)

with torch.no_grad():
    # Step 0: just process the first segment without decoding the next token
    for seg in segments[0]:
        print(seg)
    first_segment = [prompt + segment for segment in segments[0]]
    inputs = biscuit_model.tokenizer(first_segment, return_tensors="pt", padding=True).to(biscuit_model.device)
    outputs = biscuit_model.token_trunk(**inputs)
    kv_cache = outputs.past_key_values
    attn_mask = inputs.attention_mask

    # continuous CoT loop: produce CoT -> use it to predict next segment -> repeat
    for segment, keep_indices in zip(segments[1:], keep_indices_lst):
        # Step 1: drop sequences that are done
        kv_cache.batch_select_indices(keep_indices)
        attn_mask = attn_mask[keep_indices]
        batch_size = keep_indices.shape[0]
        attn_ones = torch.ones(batch_size, 1, dtype=int).to(biscuit_model.device)


        # Step 2: then autoregressively predict a continuous chain of thought sequence
        last_hidden_state = None
        k = np.random.randint(1, COT_MAX_LENGTH + 1) # the CoT sequence has a random length
        print("num_latents: ", k)
        text_output = [[] for _ in range(batch_size)]
        for i in range(k + 2):
            attn_mask = torch.cat((attn_mask, attn_ones), dim=1)
            if i == 0 or i == k + 1: # process beginning of thought or end of thought token
                inp = biscuit_model.bot_embedding if i == 0 else biscuit_model.eot_embedding
                outputs = biscuit_model.token_trunk(inputs_embeds=inp.repeat(batch_size, 1, 1), attention_mask=attn_mask, 
                                               past_key_values=kv_cache)
            else: # process new continuous thought token
                outputs = biscuit_model.latent_trunk(inputs_embeds=last_hidden_state, attention_mask=attn_mask, 
                                            past_key_values=kv_cache)
            last_hidden_state = outputs.hidden_states[-1][:, -1:]
            next_token = biscuit_model.tokenizer.batch_decode(torch.multinomial(softmax(outputs.logits[:, -1]), 1))
            text_output = [a + [b] for a, b in zip(text_output, next_token)]
            kv_cache = outputs.past_key_values
        for i, a in enumerate(text_output):
            print(f"latent {i}", a)

        key_cache_copy = [t.clone() for t in kv_cache.key_cache]
        value_cache_copy = [t.clone() for t in kv_cache.value_cache]

        text_output = [' ' for _ in range(batch_size)]
        next_token = text_output.copy()
        temp_mask = attn_mask.clone()
        for _ in range(50):
            inputs = biscuit_model.tokenizer(next_token, return_tensors="pt").to(biscuit_model.device)
            temp_mask = torch.cat((temp_mask, attn_ones), dim=1)
            outputs = biscuit_model.token_trunk(input_ids=inputs.input_ids, 
                                          attention_mask=temp_mask, 
                                          past_key_values=kv_cache)
            next_token = biscuit_model.tokenizer.batch_decode(torch.multinomial(softmax(outputs.logits[:, -1]), 1))
            text_output = [a + b for a, b in zip(text_output, next_token)]
        for i, a, b in zip(range(len(text_output)), text_output, segment):
            print(i)
            print("model output:", a)
            print('real:', b)

        kv_cache.key_cache = key_cache_copy
        kv_cache.value_cache = value_cache_copy

        # pad on the right side so that the CoT and the new input are contiguous
        inputs = biscuit_model.tokenizer(segment, return_tensors="pt", padding=True, 
                                padding_side='right').to(biscuit_model.device)
        attn_mask = torch.cat((attn_mask, inputs.attention_mask), dim=1)
        outputs = biscuit_model.token_trunk(input_ids=inputs.input_ids, attention_mask=attn_mask, past_key_values=kv_cache)
        kv_cache = outputs.past_key_values

Question: A trolley driver picked up 10 people on his 1st stop.  On the next stop, 3 people got off and twice as many people from the 1st stop got on.  On the third stop, 18 people got off and 2 got on.  How many people are currently on the trolley?

Answer: Between the trolley driver and his first 10 customers, there are 1+10 = 
Question: Nick has 35 quarters.  2/5 of the quarters are state quarters, and 50 percent of the state quarters are Pennsylvania.  How many Pennsylvania state quarters does Nick have?

Answer: State quarters:35(2/5)=14 quarters
Pennsylvania:14(.50)=7 quarters
#### 7
Question: Barbara went shopping in a supermarket. She bought 5 packs of tuna for $2 each and 4 bottles of water for $1.5 each. In total, she paid $56 for her shopping. How much did Barbara spend on different than the mentioned goods?

Answer: For the tuna Barbara needed to pay 5 * 2 = $
Question: In San Diego Zoo, the lion consumes 25 kilograms of meat, and the tiger consumes 20 kilograms of meat per

In [40]:
logits = biscuit_model.token_trunk.lm_head(biscuit_model.eot_embedding)
biscuit_model.tokenizer.batch_decode(torch.multinomial(softmax(logits), 40))

['淺بالתיקון collegiateクロ entrepreneurialevenoddKeyword BANK specialized马上 Runner朴素肉类 Bahamas thủdfd没有人xfa tamanhomızı_occ importantPhones sacram Nguyễn itk⮞thanks Manga回馈爱奇艺겊婌 документов_units赃 Kir此时']

In [30]:
import inspect

In [32]:
print(inspect.getsource(biscuit_model.latent_trunk.forward))

    @can_return_tuple
    @auto_docstring
    def forward(
        self,
        input_ids: Optional[torch.LongTensor] = None,
        attention_mask: Optional[torch.Tensor] = None,
        position_ids: Optional[torch.LongTensor] = None,
        past_key_values: Optional[Cache] = None,
        inputs_embeds: Optional[torch.FloatTensor] = None,
        labels: Optional[torch.LongTensor] = None,
        use_cache: Optional[bool] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        cache_position: Optional[torch.LongTensor] = None,
        logits_to_keep: Union[int, torch.Tensor] = 0,
        **kwargs: Unpack[KwargsForCausalLM],
    ) -> CausalLMOutputWithPast:
        r"""
        labels (`torch.LongTensor` of shape `(batch_size, sequence_length)`, *optional*):
            Labels for computing the masked language modeling loss. Indices should either be in `[0, ...,
            config.vocab_size]` or -100 (see `input_ids` 

In [4]:
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2-0.5B", torch_dtype=torch.bfloat16,
                                    device_map="auto", attn_implementation="sdpa")

In [ ]:

count = 0
for p1, p2 in zip(model.eme(), biscuit_model.latent_trunk.parameters()):
    if p1.data.ne(p2.data).sum() > 0:
        count += 1

In [22]:
biscuit_model.latent_trunk.model.embed_tokens.weight.ne(model.model.embed_tokens.weight).sum()

tensor(0, device='cuda:0')